In [2]:
import numpy as np
import matplotlib.pyplot as pl

In [3]:
train_data = np.load("dataset/train_data.npy", encoding='bytes')
train_label = np.load("dataset/train_label.npy", encoding='bytes')
test_data = np.load("dataset/test_data.npy", encoding='bytes')
test_label = np.load("dataset/test_label.npy", encoding='bytes')
train_dataset = np.hstack([train_data, train_label])
test_dataset = np.hstack([test_data, test_label])
test_dataset

array([[-3.47967057e+00,  9.06426369e-01,  1.25195572e+00, ...,
         6.31729322e-01,  2.60908392e-01,  3.00000000e+00],
       [ 9.94315784e+00, -9.58055259e+00,  5.06857801e+00, ...,
        -3.18574521e-01, -2.67055346e-01,  8.00000000e+00],
       [ 4.70429957e+00, -8.83720616e+00,  4.10928532e+00, ...,
         2.65140024e-01,  1.67865616e-01,  8.00000000e+00],
       ...,
       [-1.52911933e+01,  2.50308666e+00, -2.27169405e-01, ...,
        -6.99688324e-02, -6.08919845e-01,  5.00000000e+00],
       [-5.85707877e+00,  2.04437491e+00,  3.65488937e+00, ...,
         6.86344933e-03,  2.01131653e-02,  1.00000000e+00],
       [-1.76542944e+00, -1.89117258e+00, -2.14885246e+00, ...,
         1.87276097e-02,  2.38587672e-01,  7.00000000e+00]])

# Data Preprocessing

In [4]:
# check missing
import pandas as pd
train_df = pd.DataFrame(train_dataset)
test_df = pd.DataFrame(test_dataset)

from subprocess import TimeoutExpired
def check_missing_data(df):
    # check for any missing data in the df
    check = list(df.isnull().sum())
    miss = False
    for i in check:
        if i == 1:
            miss = True
            break
    return miss
print(check_missing_data(train_df))
print(check_missing_data(test_df))

False
False


In [5]:
print(train_df.shape)

(50000, 129)


In [6]:
print(test_df.shape)

(10000, 129)


## Standardize

In [7]:
class Standard_scaler(object):
    def __init__(self, mu = None, std = None):
        self.mu = mu
        self.std = std
    
    def fit(self, X):
        self.mu = np.mean(X, axis=0)
        self.std = np.std(X, axis=0)
        return self
    
    def transform(self, X):
        return (X - self.mu) / self.std
    
    def fit_transform(self, X):
        return self.fit(X).transform(X)
        

In [8]:
## use train mu and sd to normalize test data
scaler = Standard_scaler().fit(train_data)
x_train_norm = scaler.transform(train_data)
x_test_norm = scaler.transform(test_data)

In [9]:
print(x_train_norm.max())
print(x_train_norm.min())

13.523460647273202
-9.657574846694638


In [10]:
print(x_test_norm.max())
print(x_test_norm.min())

8.651056000808705
-8.263215822170624


## Activation

In [11]:
class Activation(object):
    def __relu(self, x):
        return np.maximum(0, x)
    
    def __relu_deriv(self, a):
       # 1 for x>=0 and 0 for x <0
       # reference: https://stackoverflow.com/questions/46411180/implement-relu-derivative-in-python-numpy
        return 1 * (a>=0)
      
    def __tanh(self, x):
        return np.tanh(x)

    def __tanh_deriv(self, a):
        # a = np.tanh(x)   
        return 1.0 - a**2
    
    def __logistic(self, x):
        return 1.0 / (1.0 + np.exp(-x))

    def __logistic_deriv(self, a):
        # a = logistic(x) 
        return  a * (1 - a)
    
    def __sigmoid(self, x):
        return 1./(1+np.exp(-x))
    
    def __sigmoid_deriv(self, a):
        return np.exp(-a)/((1+np.exp(-a))**2)
    
    def __softmax(self, x):
        # https://www.adeveloperdiary.com/data-science/deep-learning/neural-network-with-softmax-in-python/
        # unstable and alwasy get NaN result due to floating point limitation
        # a popular choice is to -max(x)
        # https://stackoverflow.com/questions/34968722/how-to-implement-the-softmax-function-in-python
        mx = np.max(x,axis=1,keepdims = True)
        x_exp = np.exp(x - mx)
        x_sum = np.sum(x_exp, axis = 1, keepdims = True)
        res = x_exp / x_sum
        """
        mx = np.max(x, axis = 1)
        e = np.exp(x - mx)
        res =  e / e.sum(axis = 1)
        """
        return res
        
    def __softmax_deriv(self, y, y_pred):
        return y_pred - y
    
    
    def __init__(self,activation='relu'):
        if activation == 'logistic':
            self.f = self.__logistic
            self.f_deriv = self.__logistic_deriv
        elif activation == 'tanh':
            self.f = self.__tanh
            self.f_deriv = self.__tanh_deriv
        elif activation == 'relu':
            self.f = self.__relu
            self.f_deriv = self.__relu_deriv
        elif activation == 'softmax':
            self.f = self.__softmax
            self.f_deriv = self.__softmax_deriv
        elif activation == 'sigmoid':
            self.f = self.__sigmoid
            self.f_deriv = self.__sigmoid_deriv
    
    def forward(self, x):
        """
        :param x: linear layer input
        """
        x_out = self.f(x)
        return x_out
    
    def backward(self, delta):
        delta = self.f_deriv(delta) * delta
        return delta

## Dropout

In [12]:
class Dropout(object):
    """
    Inverted dropout implementation of a MLP
    reference: https://blog.csdn.net/huqinweI987/article/details/103229158
    """
    def __init__(self, dropout_prob):
        self.dropout_prob = dropout_prob
        self.mask = None
        
    def forward(self, x, is_training = True):
        if is_training:
            self.mask = np.random.binomial(n=1, p = 1-self.dropout_prob, size = x.shape)
            result = x * self.mask
            return result/(1-self.dropout_prob)
        else:
            return x
    
    def backward(self, delta):
        """
        https://stats.stackexchange.com/questions/207481/dropout-backpropagation-implementation
        """
        delta = delta * self.mask/(1-self.dropout_prob)
        return delta 

## Batch Normalization

In [13]:
class Batch_Normalization(object):
    """
    
    """
    def __init__(self, epsilon = 1e-5, momentum = 0.9):
        self.epsilon = epsilon 
        self.momentum = momentum
        # for mini-batch training, need to keep a global record for mean and variance
        self.global_mean = None
        self.global_var = None
        self.X = None
        self.X_normalized = None
        self.gamma = None
        self.beta = None
        self.dgamma = None
        self.dbeta = None
        self.v_gamma = None
        self.v_beta = None
        
    def forward(self, X, is_training = True):
        """
        Batch normalization transfer for mini-batch data
        reference: 
            1. Pytorch source: https://pytorch.org/docs/stable/_modules/torch/nn/modules/batchnorm.html#BatchNorm1d
            2. https://medium.com/analytics-vidhya/deep-learning-basics-batch-normalization-ae105f9f537e
            3. https://stats.stackexchange.com/questions/219808/how-and-why-does-batch-normalization-use-moving-averages-to-track-the-accuracy-o
        """
        N, D = X.shape
        
        sample_mean = np.mean(X, axis = 0)
        sample_var = np.var(X, axis = 0)
        
        if self.global_mean is None:
            # initialize 
            self.global_mean = sample_mean
            self.global_var =  sample_var
            self.gamma = np.ones(D, dtype = X.dtype)
            self.beta = np.zeros(D, dtype = X.dtype)
            self.v_gamma = np.zeros(self.gamma.shape)
            self.v_beta = np.zeros(self.beta.shape)

        if is_training:
            # running_mean = momentum * running_mean + (1 - momentum) * sample_mean
            self.global_mean = self.momentum * self.global_mean + (1 - self.momentum) * sample_mean
            # running_var = momentum * running_var + (1 - momentum) * sample_var
            self.globar_var = self.momentum * self.global_var + (1 - self.momentum) * sample_var
            
            X_hat = (X - self.global_mean)/np.sqrt(self.global_var + self.epsilon)
            y = np.multiply(self.gamma, X_hat) + self.beta
            
        # for testing        
        else:
            X_hat = (X - self.global_mean)/np.sqrt(self.global_var + self.epsilon)
            y = np.multiply(self.gamma, X_hat) + self.beta
        
        # save X
        self.X = X
        self.X_normalized = X_hat
        
        return y
    
    def backward(self, delta):
        """
        reference: https://www.adityaagrawal.net/blog/deep_learning/bprop_batch_norm
        http://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html
        """
        N,D = delta.shape
        xmu = self.X-self.global_mean
        inverse_sqrt_var = 1/np.sqrt(self.global_var + self.epsilon) # intermediate value 
        
        self.dgamma = np.sum(delta*self.X_normalized, axis=0)
        self.dbeta = np.sum(delta, axis=0)
        
        dxhat = delta * self.gamma
        dvar = np.sum(dxhat*xmu - 0.5 * (self.global_var + self.epsilon)**(-1.5), axis=0)
        dmu = -np.sum(dxhat*inverse_sqrt_var, axis = 0) - 1./N * dvar * np.sum(2*xmu, axis = 0)
        
        delta = dxhat*inverse_sqrt_var + dvar*2*xmu/N + dmu/N
        
        return delta
    
    def update(self, optimizer, weight_decay):
        if optimizer.__class__.__name__ == 'SGD':
            self.beta = optimizer_object.update(self.beta, self.dbeta, weight_decay) #*  weight_decay
            self.gamma = optimizer_object.update(self.gamma, self.dgamma, weight_decay) #*  weight_decay


## Optimizer

In [14]:
class Optimizer(object):
    def __init__(self, lr):
        self.lr = lr
    
    def update(self):
        pass
    
class SGD(Optimizer):
    def __init__(self, lr):
        super(SGD, self).__init__(lr)
    
    def update(self, x, dfdx, weight_decay):
        x -= self.lr * dfdx
        self.lr = self.lr * weight_decay
        return x
    
class Momentum(Optimizer):
    """
    reference: https://towardsdatascience.com/neural-network-optimization-7ca72d4db3e0
    """
    def __init__(self, lr, momentum = 0.9, v = None):
        super(Momentum, self).__init__(lr)
        self.momentum = momentum
        
    def update(self, v, x, dfdx, weight_decay):
        """
        one single update for all weights
        w: weight
        grad: gradient
        """
        v = v * self.momentum - self.lr * dfdx
        x = x + v
        self.lr = self.lr * weight_decay
        return v, x 

class NAG(Optimizer):
    def __init__(self, lr, momentum = 0.9, v = None):
        super(NAG, self).__init__(lr)
        self.momentum = momentum

    def update(self, v, x, dfdx, weight_decay):
        v = v * self.momentum - self.lr * (x - self.momentum * v)
        x = x - v
        self.lr = self.lr * weight_decay
        return v, x

class Adagrad(Optimizer):
    def __init__(self, lr, epsilon = 1e-7):
        super(Adagrad, self).__init__(lr)
        self.epsilon = epsilon
        
    def update(self, v, x, dfdx, weight_decay):
        self.r += np.square(dfdx)
        dx = dfdx * self.lr/(np.sqrt(self.r) + self.epsilon)
        x = x - dx
        self.lr = self.lr * weight_decay
        return x

# class Adam(Optimizer):
#     def __init__(self, lr, rho1 = 0.9, rho2 = 0.999, epsilon = 1e-7):
#         super(Adam, self).__init__(lr)
#         self.rho1 = rho1
#         self.rho2 = rho2
#         self.epsilon = epsilon
#         self.m = None # first moment
#         self.v = None # second moment
#         self.time = 0
        
#     def update(self, x, dfdx):
#         if self.m is None:
#             self.m = np.zeros_like(x)
#             self.v = np.zeros_like(x)
            
#         self.time += 1
#         self.m = self.m * self.rho1 + (1 - self.rho1)* dfdx
#         self.v = self.v * self.rho2 + (1 - self.rho2)* (dfdx**2)
        
#         m_hat = self.m/(1-self.rho1**self.time)
#         v_hat = self.v/(1-self.rho2**self.time)
        
#         x = x - self.lr * m_hat/(np.sqrt(v_hat)+ self.epsilon)
#         return x 
            

## Linear Layer

In [15]:
class LinearLayer(object):    
    def __init__(self, n_in, n_out, W=None, b=None):
        """
        Typical hidden layer of a MLP: units are fully-connected. Weight matrix W is of shape (n_in,n_out)
        and the bias vector b is of shape (n_out,).

        NOTE : The nonlinearity used here is tanh

        Hidden unit activation is given by: tanh(dot(input,W) + b)

        :type n_in: int
        :param n_in: dimensionality of input

        :type n_out: int
        :param n_out: number of hidden units

        :type activation: string
        :param activation: Non linearity to be applied in the hidden
                           layer
        """
        
        self.input_v = None
        self.output = None 
        
        # we randomly assign small values for the weights as the initiallization
        
        np.random.seed(2022)
        self.W = np.random.uniform(
                low=-np.sqrt(6. / (n_in + n_out)),
                high=np.sqrt(6. / (n_in + n_out)),
                size=(n_in, n_out)
        )
        
        #self.W = np.ones((n_in, n_out))*4
        #self.W = np.zeros((n_in, n_out))
        #print(self.W)
         
        # we set the size of bias as the size of output dimension
        self.b = np.zeros(n_out,)
        
        # we set he size of weight gradation as the size of weight
        self.grad_W = np.zeros(self.W.shape)
        self.grad_b = np.zeros(self.b.shape)
        
        self.v_W = np.zeros(self.W.shape)
        self.v_b = np.zeros(self.b.shape)
         
    def forward(self, input_v):
        '''
        :type input_v: numpy.array
        :param input: a symbolic tensor of shape (n_in,)
        '''          
        self.input_v= input_v
        self.output = np.dot(input_v, self.W) + self.b
        return self.output
    
    def backward(self, delta): 
        self.grad_W = np.atleast_2d(self.input_v).T.dot(np.atleast_2d(delta))

        self.grad_b = np.mean(delta,axis = 0)
        delta = np.dot(delta, self.W.T)
        assert(self.grad_W.shape == self.W.shape)
        assert(self.grad_b.shape == self.b.shape)
        return delta
    
    def update(self, optimizer, weight_decay):
        if optimizer.__class__.__name__ == 'SGD':
            
            self.W = optimizer.update(self.W, self.grad_W,weight_decay) #*  weight_decay
            
            self.b = optimizer.update(self.b, self.grad_b,weight_decay) #  weight_decay     

## MLP

In [16]:
def shuffle(X, y):
    """
    https://stackoverflow.com/questions/23289547/shuffle-two-list-at-once-with-same-order
    """
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    X = X[indices]
    y = y[indices] 
    return X, y

In [17]:
import math
import time

class MLP:
    """
    """ 

    # for initiallization, the code will create all layers automatically based on the provided parameters.     
    def __init__(self, n_in, n_out, layers, optimizer = None, activation = 'relu', activation_last_layer = 'softmax',
                 dropout_ratio = 0, is_batch_normalization = False, momentum = 0.9):
        """
        :param layers: A list containing the number of units in each layer.
        Should be at least two values
        :param activation: The activation function to be used. Can be
        "logistic" or "tanh"
        """        

        # initialize layers
        self.layers=[]
        self.activation= activation
        self.optimizer = optimizer
        self.n_in = n_in
        self.n_out = n_out
        
        for i in range(len(layers)): 
            if i == 0:
                # adding the first hidden layer
                # we fix activation to be one kind to compare performance
                self.layers.append(LinearLayer(n_in,layers[i]))
            else:
                # adding the rest of hidden layers
                self.layers.append(LinearLayer(layers[i-1],layers[i]))
                
            # we assume the MLP structure is Linear -- Dropout -- Batch Normlization
            # dropout_ratio == 0 means no dropout
            if dropout_ratio != 0:
                self.layers.append(Dropout(dropout_ratio))
                
            if is_batch_normalization:
                self.layers.append(Batch_Normalization())
            
            if i!=0:
                self.layers.append(Activation(activation))
        
        # adding the output layer
        self.layers.append(LinearLayer(layers[-1], n_out))
        self.layers.append(Activation(activation_last_layer))

    # forward progress: pass the information through the layers and out the results of final output layer
    def forward(self,input_v):
        for layer in self.layers:
            output = layer.forward(input_v)
            input_v = output
        return output
   
    # backward progress  
    def backward(self,delta):
        # print(delta)
        for layer in reversed(self.layers[:-1]):
            delta=layer.backward(delta)
            
    # define the objection/loss function, we use cross_entropy as the loss
    def criterion_cross_entropy(self,y,y_hat):
        """
        y_hat: batch_size * 1
        y : batch_size * n_class
        y_actual_onehot: one hot encoding of y_hat, (batch_size * n_class)
        """
        # cross entropy
        y_actual_onehot = np.eye(self.n_out)[y].reshape(-1, self.n_out)
        
        # restrict the range into [1e-12, 1-1e-12]
        #y_predicted = np.clip(y_hat, 1e-12, 1-1e-12)
        
        #sumn = np.sum(np.exp(y_hat))
        #y_predicted = y_hat/sumn
        y_predicted = np.clip(y_hat, 1e-12, 1-1e-12)
        
        #print(y_actual_onehot.shape) (100,10)
        #print(y_predicted.shape) (100,10)
        #print(y_actual_onehot)
        #print(y_predicted)
        #print(y_actual_onehot)
        #print(y_predicted)
        
        # sum by row
        loss = - np.sum(np.multiply(y_actual_onehot, np.log(y_predicted)), axis = 1)
        # add weight decay parameter
#         weight_decay_loss = loss * weight_decay
        
        # derivative of cross entropy with softmax
        # self.layers[-1] is the activation function
        delta = self.layers[-1].f_deriv(y_actual_onehot, y_predicted)
        # return loss and delta
        return loss, delta

    # update the network weights after backward.
    def update(self,lr, weight_decay):
        if self.optimizer is None or self.optimizer == 'SGD':
            optimizer_object = SGD(lr)
            
            for layer in self.layers:
                if layer.__class__.__name__ =='LinearLayer':
                    layer.update(optimizer_object, weight_decay)
                elif layer.__class__.__name__ =='Batch_Normalization':
                    layer.update(optimizer_object, weight_decay)
#                 if layer.__class__.__name__ =='LinearLayer':
#                     layer.W = optimizer_object.update(layer.W, layer.grad_W) *  weight_decay
#                     print(layer.W)
#                     layer.b = optimizer_object.update(layer.b, layer.grad_b) *  weight_decay
#                     print(layer.b)
#                 elif layer.__class__.__name__ =='Batch_Normalization':
#                     layer.beta = optimizer_object.update(layer.beta, layer.dbeta) *  weight_decay
#                     layer.gamma = optimizer_object.update(layer.gamma, layer.dgamma) *  weight_decay
        
        elif self.optimizer == 'Momentum':
            optimizer_object = Momentum(lr)
            for layer in self.layers:
                if layer.__class__.__name__ =='LinearLayer':
                    layer.v_W, layer.W = optimizer_object.update(layer.v_W, layer.W, layer.grad_W, weight_decay)
                    layer.v_b, layer.b = optimizer_object.update(layer.v_b, layer.b, layer.grad_b, weight_decay)

                elif layer.__class__.__name__ =='Batch_Normalization':
                    layer.v_gamma, layer.gamma = optimizer_object.update(layer.v_gamma, layer.gamma, layer.dgamma, weight_decay)
                    layer.v_beta, layer.beta = optimizer_object.update(layer.v_beta, layer.beta, layer.dbeta, weight_decay)
        '''
        elif self.optimizer == 'NAG':
            optimizer_object = NAG(lr)
            for layer in self.layers:
                if layer.__class__.__name__ =='LinearLayer':
                    layer.v_W, layer.W = optimizer_object.update(layer.v_W, layer.W, layer.grad_W)

                    layer.v_b, layer.b = optimizer_object.update(layer.v_b, layer.b, layer.grad_b)

                elif layer.__class__.__name__ =='Batch_Normalization':
                    layer.v_gamma, layer.gamma = optimizer_object.update(layer.v_gamma, layer.gamma, layer.dgamma)
                    layer.v_beta, layer.beta = optimizer_object.update(layer.v_beta, layer.beta, layer.dbeta)
        '''                    
                
#        elif self.optimizer == 'Adagrad':
#            optimizer_object = Adagrad(lr)
#        elif self.optimizer == 'Adam':
#            optimizer_object = Adam(lr)

        for layer in self.layers:
            if layer.__class__.__name__ =='LinearLayer':
                if self.optimizer is None or self.optimizer == 'SGD':
                    optimizer_object = SGD(lr)
                    layer.W = optimizer_object.update(layer.W, layer.grad_W,weight_decay)
                    layer.b = optimizer_object.update(layer.b, layer.grad_b,weight_decay)

                elif self.optimizer == 'Momentum':
                    optimizer_object = Momentum(lr)
                    layer.v_W, layer.W = optimizer_object.update(layer.v_W, layer.grad_W, layer.W,weight_decay)
                    layer.v_b, layer.b = optimizer_object.update(layer.v_b, layer.grad_b, layer.b,weight_decay)
                    
    # define the training function
    # it will return all losses within the whole training process.
    def fit(self, X,y, is_shuffle = False, learning_rate=0.001, epochs=100, batch_size = 100, weight_decay = 0.95):
        """
        Online learning with mini-batch training.
        :param X: Input data or features
        :param y: Input targets
        :param learning_rate: parameters defining the speed of learning
        :param epochs: number of times the dataset is presented to the network for learning
        """ 
        X = np.array(X)
        y = np.array(y)
    
        # initialize loss array for epoch training 
        to_return = np.zeros(epochs)

        n_batch = math.ceil(X.shape[0] / batch_size)
        
        batch_loss = np.zeros(n_batch)
                
        for k in range(epochs):
            time_start = time.time()
            y_pred = None
            
            if is_shuffle == True:
                X, y = shuffle(X, y)
            
    
            #initialize loss array for mini-batch training 
            batch_loss = np.zeros(n_batch)
            
            for j in range(n_batch):
                
                # decide samples for each mini-batch
                if j != n_batch-1:
                    X_batch = X[j*batch_size:(j+1)*batch_size]
                    y_batch = y[j*batch_size:(j+1)*batch_size]
                else:
                    X_batch = X[j*batch_size:]
                    y_batch = y[j*batch_size:]
                    
                # forward pass
                y_batch_hat = self.forward(X_batch)
                
                sumn = np.sum(np.exp(y_batch_hat))
                y_batch_hat = (np.exp(y_batch_hat)/sumn)*9

                # calculate loss and backward pass
                loss, delta = self.criterion_cross_entropy(y_batch, y_batch_hat)
                self.backward(delta)
                # update
                self.update(learning_rate, weight_decay)
                
                # calculate loss and accuracy 
                batch_loss[j] = np.sum(loss)

                #after argmax
                y_batch_pred = np.argmax(y_batch_hat, axis = 1).reshape(-1,1)
                print(y_batch_pred)
#                 print(y_batch_hat)
#                 print(y_batch_pred) 
                if y_pred is None:
                    y_pred = y_batch_pred
                else:
                    y_pred = np.vstack((y_pred, y_batch_pred))
                    
            epoch_loss = np.mean(batch_loss)
            
            to_return[k] = epoch_loss
            
            # for every epoch, print time and loss
            accuracy = np.sum(y_pred == y) / y.shape[0]
            print("epoch {} loss {:.6f}, accuracy {:.6f}% ".format(k, epoch_loss, 100*accuracy))
        return to_return
    
    # define the prediction function
    # we can use predict function to predict the results of new data, by using the well-trained network.
    def predict(self, x):
        x = np.array(x)
        output = np.zeros(x.shape[0])
        for i in np.arange(x.shape[0]):
            output[i] = self.forward(x[i,:])
        return output


In [18]:
train_label.max()

9

In [19]:
train_label.min()

0

## Training

In [34]:
n_input_training = x_train_norm.shape[1]
n_class = len(np.unique(test_label.reshape(1, -1)))
layers = [50, 70]
optimizer = 'SGD'
nn = MLP(n_input_training, n_class, layers, optimizer, activation = 'relu', activation_last_layer = 'softmax',
                 dropout_ratio = 0, is_batch_normalization = False)
nn.fit(x_train_norm, train_label)

epoch 0 loss 468.849447, accuracy 15.918000% 
epoch 1 loss 469.395093, accuracy 16.716000% 
epoch 2 loss 469.621436, accuracy 16.784000% 
epoch 3 loss 469.731402, accuracy 16.802000% 
epoch 4 loss 469.791107, accuracy 16.840000% 
epoch 5 loss 469.827778, accuracy 16.842000% 
epoch 6 loss 469.852797, accuracy 16.836000% 
epoch 7 loss 469.871510, accuracy 16.844000% 
epoch 8 loss 469.886435, accuracy 16.856000% 
epoch 9 loss 469.898785, accuracy 16.868000% 
epoch 10 loss 469.909174, accuracy 16.872000% 
epoch 11 loss 469.917954, accuracy 16.878000% 
epoch 12 loss 469.925385, accuracy 16.876000% 
epoch 13 loss 469.931695, accuracy 16.878000% 
epoch 14 loss 469.937091, accuracy 16.880000% 
epoch 15 loss 469.941759, accuracy 16.884000% 
epoch 16 loss 469.945854, accuracy 16.890000% 
epoch 17 loss 469.949502, accuracy 16.892000% 
epoch 18 loss 469.952803, accuracy 16.896000% 
epoch 19 loss 469.955825, accuracy 16.892000% 
epoch 20 loss 469.958621, accuracy 16.886000% 
epoch 21 loss 469.96122

array([468.84944653, 469.39509314, 469.62143593, 469.73140189,
       469.79110723, 469.82777828, 469.85279695, 469.87150957,
       469.88643508, 469.89878538, 469.90917394, 469.9179537 ,
       469.92538468, 469.93169477, 469.93709139, 469.94175876,
       469.94585351, 469.94950238, 469.95280261, 469.95582533,
       469.95862091, 469.96122448, 469.96366019, 469.96594458,
       469.96808931, 469.97010368, 469.97199624, 469.97377539,
       469.97544942, 469.97702628, 469.97851334, 469.97991733,
       469.98124431, 469.98249969, 469.98368831, 469.98481446,
       469.98588205, 469.98689457, 469.98785524, 469.98876701,
       469.98963264, 469.99045472, 469.99123566, 469.99197778,
       469.99268328, 469.99335426, 469.99399272, 469.99460059,
       469.9951797 , 469.99573181, 469.9962586 , 469.99676163,
       469.99724242, 469.99770238, 469.99814283, 469.99856501,
       469.99897009, 469.99935916, 469.99973322, 470.00009322,
       470.00044004, 470.00077449, 470.00109734, 470.00